In [1]:
import pandas as pd

In [2]:
commits = pd.read_pickle('../../data/01_data_from_db/commits.pkl')
commits.head()

,commit_id,committer_id,project_id,created_at
0,1,17,10,2012-04-08 02:29:18
1,2,18,10,2011-12-29 21:12:27
2,3,18,10,2011-12-29 21:11:10
3,4,17,10,2011-12-29 20:54:15
4,5,18,10,2011-12-29 20:29:49


In [3]:
commits['year'] = commits['created_at'].dt.year
commits['month'] = commits['created_at'].dt.month
commits.head()

,commit_id,committer_id,project_id,created_at,year,month
0,1,17,10,2012-04-08 02:29:18,2012,4
1,2,18,10,2011-12-29 21:12:27,2011,12
2,3,18,10,2011-12-29 21:11:10,2011,12
3,4,17,10,2011-12-29 20:54:15,2011,12
4,5,18,10,2011-12-29 20:29:49,2011,12


## Nowe commity w miesiącu

In [4]:
new_commits = commits.groupby(['project_id', 'year', 'month']) \
    .count().reset_index().rename(columns={'created_at': 'new_commits'}) \
    .drop(columns={'commit_id', 'committer_id'}, axis=1)
new_commits

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


Wersja alternatywna

In [5]:
commits.groupby(by= ['project_id', 'year', 'month'], as_index=False) \
    .agg({'created_at': pd.Series.nunique}) \
    .rename(columns={'created_at': 'new_commits'})

,project_id,year,month,new_commits
0,2,2010,5,43
1,2,2010,6,4
2,2,2010,7,3
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,2
18758,90366,2013,9,13
18759,91413,2013,9,1
18760,91417,2013,9,1


## Dołaczenie łącznej liczby commitów od początku projektu (uwaga - długie wykonanie)

In [6]:
total_commits_list = []
for index, row in new_commits.iterrows():
    total_commits_list.append(new_commits[(((row['year'] > new_commits['year']) |
        ((row['year'] == new_commits['year']) & (row['month'] >= new_commits['month']))) &
        (row['project_id'] == new_commits['project_id']))]['new_commits'].sum())

In [7]:
new_commits['total_commits'] = pd.Series(total_commits_list)

Sprawdzenie dla jednego z projektów

In [8]:
new_commits[new_commits['project_id'] == 3]

,project_id,year,month,new_commits,total_commits
36,3,2010,8,17,17
37,3,2010,9,1,18
38,3,2010,10,1,19
39,3,2010,11,5,24
40,3,2010,12,2,26
41,3,2011,2,2,28
42,3,2011,3,2,30
43,3,2011,6,18,48
44,3,2011,7,23,71
45,3,2011,11,3,74


In [9]:
new_commits.to_pickle('../../data/02_prepared_data/new_commits.pkl')

## Liczba unikalnych commitujących w miesiącu

In [10]:
unique_committers = commits.groupby(by = ['project_id', 'year', 'month'], as_index=False) \
    .agg({'committer_id': pd.Series.nunique}) \
    .rename(columns={'committer_id': 'unique_committers'})
unique_committers

,project_id,year,month,unique_committers
0,2,2010,5,1
1,2,2010,6,1
2,2,2010,7,1
3,2,2010,8,1
4,2,2010,9,1
...,...,...,...,...
18757,90343,2013,9,1
18758,90366,2013,9,1
18759,91413,2013,9,1
18760,91417,2013,9,1


## Liczba wszystkich unikalnych commitujących od początku projektu

- dla każdego wiersza `new_commits`
- lista `committer_id` z `commits`, gdzie:
    - `project_id` jest równe
    - rok utworzenia commita jest mniejszy niż rok w projekcie LUB (rok utworznia commita jest taki sam jak rok w projekcie ORAZ miesiąc jest mniejszy)
- dla każdego wiersza `new_commits` - agregacja, ilość unikalnych `committer_id`

In [11]:
total_unique_committers = []
for index, row in new_commits.iterrows():
    total_unique_committers.append(commits[((commits['project_id'] == row['project_id']) &
        ((commits['year'] < row['year']) | 
        ((commits['year'] == row['year']) & (commits['month'] <= row['month']))))]['committer_id'].nunique())

In [12]:
unique_committers['total_unique_committers'] = pd.Series(total_unique_committers)

Sprawdzenie dla jednego z projektów

In [13]:
unique_committers[unique_committers['project_id'] == 3]

,project_id,year,month,unique_committers,total_unique_committers
36,3,2010,8,2,2
37,3,2010,9,1,2
38,3,2010,10,1,2
39,3,2010,11,1,2
40,3,2010,12,1,2
41,3,2011,2,1,3
42,3,2011,3,1,3
43,3,2011,6,1,3
44,3,2011,7,1,3
45,3,2011,11,1,3


In [14]:
unique_committers.to_pickle('../../data/02_prepared_data/unique_committers.pkl')